# STA 141B Assignment 3

Due __Feb 11, 2019__ by 11:59pm. Submit by editing this file, committing the changes with git, and then pushing to your private GitHub repo for the assignment. This assignment will be graded according to the class rubric.

Please do not rename this file or delete the exercise cells, because it will interfere with our grading tools. Put your answers in new cells after each exercise. You can make as many new cells as you like. Use code cells for code and Markdown cells for text. Answer all questions with complete sentences.

The purpose of this assignment is to practice using web APIs and combining data from multiple sources.

## USDA Food Composition

The US Department of Agriculture publishes price estimates for fruits and vegetables [online](https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices/fruit-and-vegetable-prices/). The most recent estimates are based on a 2013 survey of US retail stores. We've cleaned and merged the fruit and vegetable price estimates for you. The result is the `fresh.csv` file included in the repository.

In this assignment, you'll combine the price estimates with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list), and then analyze the resulting dataset.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it's a good idea to use caching.

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any data.gov API. You may need the key again later in the quarter, so make sure you save it.

In [16]:
import numpy as np
import pandas as pd
import requests
import requests_cache
import time
requests_cache.install_cache('hw3_cache')

In [3]:
# read the data:
fresh = pd.read_csv('fresh.csv')
fresh.head()

,form,price_per_lb,yield,lb_per_cup,price_per_cup,food,type
0,Fresh1,0.316639,0.52,0.330693,0.201366,watermelon,fruit
1,Fresh1,0.520794,0.51,0.374786,0.382718,cantaloupe,fruit
2,Fresh1,1.480396,0.74,0.407855,0.815929,tangerines,fruit
3,Fresh1,2.508450,0.94,0.319670,0.853060,strawberries,fruit
4,Fresh1,1.987999,0.94,0.363763,0.769319,plums,fruit


__Exercise 1.1__

Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` might return this list:

```python
[{'offset': 0,
  'group': 'Branded Food Products Database',
  'name': 'EL COMPI, QUAIL EGGS, UPC: 854955002226',
  'ndbno': '45362205',
  'ds': 'LI',
  'manu': "milly's desserts llc"},
 {'offset': 1,
  'group': 'Branded Food Products Database',
  'name': 'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  'ndbno': '45099560',
  'ds': 'LI',
  'manu': 'Sung Ly International Corporation'},
 {'offset': 2,
  'group': 'Branded Food Products Database',
  'name': 'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  'ndbno': '45169279',
  'ds': 'LI',
  'manu': 'L & M C Farms, Inc.'},
 {'offset': 3,
  'group': 'Branded Food Products Database',
  'name': 'L&W, QUAIL EGGS, UPC: 024072000256',
  'ndbno': '45094890',
  'ds': 'LI',
  'manu': 'L&W International Co.'},
 {'offset': 4,
  'group': 'Branded Food Products Database',
  'name': 'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  'ndbno': '45094707',
  'ds': 'LI',
  'manu': 'Theppadung Porn Coconut Co'},
 {'offset': 5,
  'group': 'Dairy and Egg Products',
  'name': 'Egg, quail, whole, fresh, raw',
  'ndbno': '01140',
  'ds': 'SR',
  'manu': 'none'}]
```

As usual, make sure you document and test your function.

In [42]:
# define my loading key function
def read_key(x):
    with open(x) as file:
        return file.readline().strip('\n')
 
# read my key
key = read_key('C:\\Users\\RockyYin\\Documents\\Py_For_STA141B\\USDA_key.txt')
        

In [70]:
response = requests.get('https://api.nal.usda.gov/ndb/search', params={
    'api_key': key,
    'q': 'kkkk'
    })
response.json()
    


{'errors': {'error': [{'status': 400,
    'parameter': 'results',
    'message': 'Your search resulted in zero results.Change your parameters and try again'}]}}

In [63]:
def ndb_search(term):
    response = requests.get('https://api.nal.usda.gov/ndb/search', params={
    'api_key': key,
    'q': term
    })
    
    
    rs = response.json()['list']['item']
    if len(rs) == 0:
        a = [1,2,3]
        return a 
    else:
        return rs
    
    
ndb_search('shit')

KeyError: 'list'

__Exercise 1.2__

Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* The [Python](https://docs.python.org/3/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/user_guide/text.html) string methods are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

In [50]:
terms = fresh['food']
terms
pd.DataFrame(ndb_search(terms[0]))

,ds,group,manu,name,ndbno,offset
0,LI,Branded Food Products Database,"EVOLUTION FRESH, INC.",EVOLUTION FRESH RASPBERRY WATERMELON WATERMELO...,45235649,0
1,LI,Branded Food Products Database,The Minute Maid Company,"WATERMELON PUNCH NATURAL WATERMELON FLAVOR, UP...",45292992,1
2,LI,Branded Food Products Database,none,"MY FIZZ, GREEN WATERMELON SODA, GREEN WATERMEL...",45137746,2
3,GDSN,Branded Food Products Database,Mondelez Int. US (0074819091009),SOUR PATCH WATERMELON SOFT CANDY WATERMELON FA...,45368431,3
4,GDSN,Branded Food Products Database,Mondelez Int. US (0074819091009),SOUR PATCH WATERMELON SOFT CANDY WATERMELON FA...,45368546,4
5,GDSN,Branded Food Products Database,Mondelez Int. US (0074819091009),SOUR PATCH WATERMELON SOFT CANDY WATERMELON FA...,45369325,5
6,GDSN,Branded Food Products Database,Mondelez Int. US (0074819091009),SOUR PATCH WATERMELON SOFT CANDY WATERMELON FA...,45369561,6
7,LI,Branded Food Products Database,"DVC Industries, Inc","JUICERA, WATERMELON FRESCA, WATERMELON, LEMON,...",45205861,7
8,GDSN,Branded Food Products Database,Mondelez Int. US (0074819091009),SOUR PATCH WATERMELON SOFT CANDY WATERMELON FA...,45375721,8
9,GDSN,Branded Food Products Database,Mondelez Int. US (0074819091009),SOUR PATCH WATERMELON SOFT CANDY WATERMELON FA...,45375742,9


__Exercise 1.3__

Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{'nutrient_id': '255',
 'name': 'Water',
 'derivation': 'NONE',
 'group': 'Proximates',
 'unit': 'g',
 'value': '87.23',
 'measures': [{'label': 'cup, sliced',
   'eqv': 165.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '143.93'},
  {'label': 'fruit (2-1/8" dia)',
   'eqv': 66.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '57.57'},
  {'label': 'NLEA serving',
   'eqv': 151.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '131.72'}]}
```

Be sure to document and test your function.

__Exercise 1.4.__

Answer any two of the questions below. For each answer, state the question, write 2-3 paragraphs, and support your conclusions with plots.

 If you like, you can replace one question with a question you came up with on your own.

* Are fruits typically higher in kilocalories, or vegetables?
* How does yield for fibrous foods (high in dietary fiber) compare to other foods?
* Which food provides the widest variety of nutrients?
* Which nutrients are the most expensive?
* Which foods are the best value for the price? "Best value" might mean highest yield per dollar, most kilocalories per dollar, or some other reasonable definition.
* How do the price distributions compare for fruits and vegetables?